In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pandas_bokeh
# import finplot as fplt

import polygon_rest_api as papi
import polygon_backfill as pb
# import ray_backfill as rb
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl

# import filters as ft
# import market_cluster as mc

import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20, 7]
from matplotlib import style
style.use('fivethirtyeight')

# import ipdb
# ipdb.set_trace(context=10)

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG' are other good diverging colormaps
# pd.set_option('precision', 2)

# get local market daily data
# mdf = pd.read_parquet('data/mdf.parquet')

# .plot_bokeh(sizing_mode="scale_height")

In [8]:
symbol='GLD'
# symbol = 'market'
date = '2020-01-02'
start_date = '2020-01-01' 
end_date = '2020-01-10'
symbols=['SPY', 'QQQ']
tick_type='trades'
# tick_type = None

result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

In [9]:
open_dates = pb.get_open_market_dates(start_date, end_date)

In [ ]:
thresh = {
    # GT thresholds
    'renko_size': 0.2,
    'renko_reveral_multiple': 2,
    'duration_sec': 60*30,
    'tick_imbalance': 150,
    # LT thresholds 
    'min_duration_sec': 15,
    'min_tick_count': 30,
}
bar_dates = []
bar_dates_d = {}
for date in open_dates:
    
    tdf = ps3.load_ticks(result_path, symbol, date)
    
    bars, state = bs.build_bars(tdf, thresh)
    bar_dates.append(bars)
    bar_dates_d[date] = bars

In [48]:
end_price = bar_dates[0][-1]['price_wmean']
start_price = bar_dates[1][0]['price_wmean']

In [ ]:
def fill_gap_renko(start_price, end_price, renko_size):
    

In [50]:
start_price

145.68966501212864

In [53]:
end_price

143.99282257581496

In [52]:
(start_price - end_price) / renko_size

8.484212181568438

In [ ]:
start_price +

In [41]:
# sample ticks -> bars
renko_size = 0.2
# renko_reversal = 2
thresh = {
    # GT thresholds
    'renko_size': renko_size,
    'renko_reveral_multiple': 2,
    'duration_sec': 60*30,
    'tick_imbalance': 150,
    # LT thresholds 
    'min_duration_sec': 15,
    'min_tick_count': 30,
}
# ticks_df['epoch'] = ticks_df.date_time.astype(int)
tdf = ticks_df.loc[(ticks_df.date_time.dt.hour >= 13) & (ticks_df.date_time.dt.hour <= 22)]
bars, _ = bs.build_bars(ticks_df=tdf, thresh=thresh)
len(bars)

bars_df = pd.DataFrame(bars).set_index('close_at')
# bars_df = pd.DataFrame(bars)

NameError: name 'ticks_df' is not defined

In [ ]:
# range / price = HFT profit potential
(df.price.max() - df.price.min()) / df.price.mean()

In [ ]:
# label bars
label_horizon_mins = 30
risk_level = 0.2
# reward_base = 1
reward_ratios = np.arange(3, 20, .5)

lbars = bl.label_bars(bars, ticks_df, risk_level, label_horizon_mins, reward_ratios)

lbars_df = pd.DataFrame(lbars)

lbars_unq = bl.get_concurrent_stats(lbars_df)
print(lbars_unq['grand_avg_unq'])

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot_bokeh(sizing_mode="scale_height")
# 17, 31

In [ ]:
lbars_df[['price_wmean', 'label_rrr']].plot(subplots=True, figsize=[17,12])

In [ ]:
label_prices, price_end_at = bl.get_label_ticks(tdf, bars[31]['close_at'], 30)